In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_pickle('../data/out/data.pkl')

In [3]:
# Years with less than 10 movies
year_movie_count = df.date.dt.year.astype('Int64').value_counts()
year_movie_count[year_movie_count < 10].sort_index()

1893    1
1894    2
1895    9
1896    6
1897    4
1898    7
1899    1
1900    6
1901    2
1902    3
1903    4
1904    1
1905    3
1906    1
1907    8
1908    6
1909    5
2014    4
Name: date, dtype: Int64

The tf-idf with sublinear tf scaling for a term $t$ of a document $d$ is given by

$$\text{tf-idf}(t, d) = (1 + \log\text{tf}(t, d)) \cdot \left(\log\frac{1 + n}{\text{1 + df}(t)} + 1\right)$$

where $n$ is the total number of documents and $\text{df}(t)$ is the document frequency of $t$ ([source](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)).

We use sublinear tf scaling to reduce the significance of very common terms which are likely uninformative.

Here, a term is a phrase and a document is a year. Given a tf-idf vector for each year, we then normalize each tf-idf vector to have a Euclidean norm of 1.

In [4]:
phrases_by_year = df.query('1909 < date.dt.year < 2014').groupby(df.date.dt.year.astype('Int64')).phrases.agg(lambda x: sum(x, []))
phrase_counts = pd.DataFrame(phrases_by_year.apply(Counter).tolist(), index=phrases_by_year.index).fillna(0).sort_index(1)
phrase_counts = phrase_counts.loc[:, phrase_counts.sum().ge(10)]
phrases_tfidf = pd.DataFrame(TfidfTransformer(sublinear_tf=True).fit_transform(phrase_counts).toarray(), index=phrase_counts.index, columns=phrase_counts.columns)

In [5]:
phrases_tfidf.apply(lambda x: x.nlargest().index.values, 1)

date
1910    [bushranger, aboriginal, ebenezer scrooge, rac...
1911    [bushranger, oxford university, feature film, ...
1912    [national library, oxford university, feature ...
1913    [oxford university, feature film, new south wa...
1914    [chaplin, charlie chaplin, ambrose, illegitima...
                              ...                        
2009    [heart attack, internet, black market, video, ...
2010    [heart attack, internet, fashion designer, int...
2011    [heart attack, post-credits scene, santhanam, ...
2012    [heart attack, viral, facebook, software engin...
2013    [film, smithsonian, true life, upper-class, foal]
Length: 104, dtype: object